# Homework 8
## Due Date:  Tuesday, October 31st at 11:59 PM

# Problem 1:  BST Traversal
This problem builds on Problem 1 of Homework 7 in which you wrote a binary search tree.

### Part 1

As discussed in lecture, three different types to do a depth-first traversal are: preorder, inorder, and postorder. Here is a reference: [Tree Traversal](https://en.wikipedia.org/wiki/Tree_traversal#Depth-first_search).

Write an iterator class called `DFSTraversal` with the following specifications:

* `__init__(self, tree, traversalType)`: Constructor takes a `BinaryTree` object and one of the enums from `DFSTraversalTypes`

```python
from enum import Enum

class DFSTraversalTypes(Enum):
    PREORDER = 1
    INORDER = 2
    POSTORDER = 3
```

* `changeTraversalType(self, traversalType)`: Change the traversal type
* `__iter__(self)`: This is the initialization of an iterator
* `__next__(self)`: This is called in the iterator for getting the next value

Here's how you might use your `DFSTraversal` class:

```python
input_array = [3, 9, 2, 11]
bt = BinaryTree()
for val in input_array:
    bt.insert(val)
traversal = DFSTraversal(bt, DFSTraversalTypes.INORDER)
for val in traversal:
    print(val)
2
3
9
11
```

### Part 2
Put your `BinaryTree` class (from homework 7) and your `DFSTraversal` class (from Part 1 of this homework) in a file titled `TreeTraversal.py`.

In [1]:
# Part 1

from enum import Enum

class DFSTraversalTypes(Enum):
    PREORDER = 1
    INORDER = 2
    POSTORDER = 3
    
class Node:
    def __init__(self, value, depth=0, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        self.parent = None
        self.depth = depth
        
    def set_left(self, left):
        self.left = left
        if left is not None:
            left.parent = self
            left.set_depth(self.depth + 1)
        
    def set_right(self, right):
        self.right = right
        if right is not None:
            right.parent = self
            right.set_depth(self.depth + 1)
        
    def set_parent(self, parent):
        self.parent = parent
        
    def set_depth(self, depth):
        self.depth = depth
        
    def __eq__(self, other):
        return self.value == other.value
    
    def __lt__(self, other):
        return self.value < other.value
    
    def __le__(self, other):
        return self.value <= other.value
    
    def __ne__(self, other):
        return self.value != other.value
    
    def __gt__(self, other):
        return self.value > other.value
    
    def __ge__(self, other):
        return self.value >= other.value

class BinaryTree:
    def __init__(self):
        self.root = None
    
    def insert(self, val):
        new_node = Node(val)
        if self.root is None:
            self.root = new_node
            return
        curr = self.root
        while True:
            if curr < new_node:
                if curr.right is None:
                    curr.set_right(new_node)
                    return
                else:
                    curr = curr.right
                    continue
            elif new_node < curr:
                if curr.left is None:
                    curr.set_left(new_node)
                    return
                else:
                    curr = curr.left
                    continue
            elif new_node == curr:
                new_node.set_left(curr.left)
                curr.set_left(new_node)
                # update depth for the subtree
                self.getInOrder(new_node)
                return
    
    def remove(self, val):
        curr = self.root
        from_ = None
        while True:
            if val < curr.value:
                curr = curr.left
                from_ = 1
                if curr is None:
                    return
                continue
            elif curr.value < val:
                curr = curr.right
                from_ = 2
                if curr is None:
                    return
                continue
            else:
                if curr.left is None and curr.right is None:
                    if from_ is None:
                        self.root = None
                    elif 1 == from_:
                        curr.parent.set_left(None)
                    else:
                        curr.parent.set_right(None)
                elif curr.left is not None and curr.right is None:
                    if from_ is None:
                        self.root = curr.left
                        self.root.set_parent(None)
                        self.root.set_depth(0)
                        # update depth for the subtree
                        self.getInOrder(self.root)
                    elif 1 == from_:
                        curr.parent.set_left(curr.left)
                        # update depth for the subtree
                        self.getInOrder(curr.parent)
                    else:
                        curr.parent.set_right(curr.left)
                        # update depth for the subtree
                        self.getInOrder(curr.parent)
                elif curr.left is None and curr.right is not None:
                    if from_ is None:
                        self.root = curr.right
                        self.root.set_parent(None)
                        self.root.set_depth(0)
                        # update depth for the subtree
                        self.getInOrder(self.root)
                    elif 1 == from_:
                        curr.parent.set_left(curr.right)
                        # update depth for the subtree
                        self.getInOrder(curr.parent)
                    else:
                        curr.parent.set_right(curr.right)
                        # update depth for the subtree
                        self.getInOrder(curr.parent)
                else:
                    # find the node (`r_node`) with the maximum value in the left subtree
                    r_node = curr.left
                    if r_node.right is None:
                        is_left = True
                    else:
                        is_left = False
                    while r_node.right is not None:
                        r_node = r_node.right
                    if is_left:
                        r_node.set_right(curr.right)
                    else:
                        r_node.parent.set_right(r_node.left)
                        r_node.set_left(curr.left)
                        r_node.set_right(curr.right)
                    if from_ is None:
                        self.root = r_node
                        self.root.set_parent(None)
                        self.root.set_depth(0)
                        # update depth for the subtree
                        self.getInOrder(self.root)
                    elif 1 == from_:
                        curr.parent.set_left(r_node)
                        # update depth for the subtree
                        self.getInOrder(curr.parent)
                    else:
                        curr.parent.set_right(r_node)
                        # update depth for the subtree
                        self.getInOrder(curr.parent)
                break
        self.remove(val)
                            
    def getValues(self, depth):
        inorder = self.getInOrder(self.root)
        values = []
        for node in inorder:
            if node.depth > depth:
                continue
            elif node.depth == depth:
                values.append(node.value)
            else:
                if node.left is None:
                    values += [None] * 2**(depth-node.depth-1)
                if node.right is None:
                    values += [None] * 2**(depth-node.depth-1)
        return values
    
    def getInOrder(self, node):
        # in-order traversal
        # update depth
        inorder = []
        def _inorder(node, inorder):
            if node is None:
                return
            elif node.parent is not None:
                node.set_depth(node.parent.depth + 1)
            _inorder(node.left, inorder)
            inorder.append(node)
            _inorder(node.right, inorder)
        _inorder(node, inorder)
        return inorder
    
    def maxDepth(self):
        inorder = self.getInOrder(self.root)
        return max([i.depth for i in inorder])
    
    def __len__(self):
        return self.maxDepth() + 1
    
class DFSTraversal:
    def __init__(self, tree, traversalType):
        self.tree = tree
        self.traversalType = traversalType
        self.changeTraversalType(traversalType)
    
    def changeTraversalType(self, traversalType):
        if not traversalType in DFSTraversalTypes:
            raise TypeError('Invalid traversalType')
        self.traversalType = traversalType
        if traversalType == DFSTraversalTypes.PREORDER:
            self.traversal = self.getPreOrder(self.tree.root)
        elif traversalType == DFSTraversalTypes.INORDER:
            self.traversal = self.getInOrder(self.tree.root)
        elif traversalType == DFSTraversalTypes.POSTORDER:
            self.traversal = self.getPostOrder(self.tree.root)
    
    def getPreOrder(self, node):
        traversal = []
        if node is None:
            return traversal
        s = []
        s.append(node)
        while len(s) > 0:
            node = s.pop()
            traversal.append(node.value)
            if node.right is not None:
                s.append(node.right)
            if node.left is not None:
                s.append(node.left)
        return traversal
    
    def getInOrder(self, node):
        traversal = []
        s = []
        while len(s) > 0 or node is not None:
            if node is not None:
                s.append(node)
                node = node.left
            else:
                node = s.pop()
                traversal.append(node.value)
                node = node.right
        return traversal
    
    def getPostOrder(self, node):
        traversal = []
        s = []
        last_node_visited = None
        while len(s) > 0 or node is not None:
            if node is not None:
                s.append(node)
                node = node.left
            else:
                peek_node = s[-1]
                if peek_node.right is not None and id(last_node_visited) != id(peek_node.right):
                    node = peek_node.right
                else:
                    traversal.append(peek_node.value)
                    last_node_visited = s.pop()
        return traversal
    
    def __getitem__(self, i):
        return self.traversal[i]
        
    def __iter__(self):
        self.index = 0
        return self
    
    def __next__(self):
        try:
            elem = self.traversal[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return elem

In [2]:
# example

input_array = [3, 9, 2, 11]
bt = BinaryTree()
for val in input_array:
    bt.insert(val)
traversal = DFSTraversal(bt, DFSTraversalTypes.PREORDER)
print('pre-oder:')
for val in traversal:
    print(val)
print()

traversal.changeTraversalType(DFSTraversalTypes.INORDER)
print('in-order:')
for val in traversal:
    print(val)
print()

traversal.changeTraversalType(DFSTraversalTypes.POSTORDER)
print('post-order:')
for val in traversal:
    print(val)
print()

print('test iterator:')
t_it = iter(traversal)
while True:
    try:
        print(next(t_it))
    except StopIteration:
        del t_it
        break

pre-oder:
3
2
9
11

in-order:
2
3
9
11

post-order:
2
11
9
3

test iterator:
2
11
9
3


In [3]:
# Part 2

In [4]:
%%file TreeTraversal.py

from enum import Enum

class DFSTraversalTypes(Enum):
    PREORDER = 1
    INORDER = 2
    POSTORDER = 3
    
class Node:
    def __init__(self, value, depth=0, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        self.parent = None
        self.depth = depth
        
    def set_left(self, left):
        self.left = left
        if left is not None:
            left.parent = self
            left.set_depth(self.depth + 1)
        
    def set_right(self, right):
        self.right = right
        if right is not None:
            right.parent = self
            right.set_depth(self.depth + 1)
        
    def set_parent(self, parent):
        self.parent = parent
        
    def set_depth(self, depth):
        self.depth = depth
        
    def __eq__(self, other):
        return self.value == other.value
    
    def __lt__(self, other):
        return self.value < other.value
    
    def __le__(self, other):
        return self.value <= other.value
    
    def __ne__(self, other):
        return self.value != other.value
    
    def __gt__(self, other):
        return self.value > other.value
    
    def __ge__(self, other):
        return self.value >= other.value

class BinaryTree:
    def __init__(self):
        self.root = None
    
    def insert(self, val):
        new_node = Node(val)
        if self.root is None:
            self.root = new_node
            return
        curr = self.root
        while True:
            if curr < new_node:
                if curr.right is None:
                    curr.set_right(new_node)
                    return
                else:
                    curr = curr.right
                    continue
            elif new_node < curr:
                if curr.left is None:
                    curr.set_left(new_node)
                    return
                else:
                    curr = curr.left
                    continue
            elif new_node == curr:
                new_node.set_left(curr.left)
                curr.set_left(new_node)
                # update depth for the subtree
                self.getInOrder(new_node)
                return
    
    def remove(self, val):
        curr = self.root
        from_ = None
        while True:
            if val < curr.value:
                curr = curr.left
                from_ = 1
                if curr is None:
                    return
                continue
            elif curr.value < val:
                curr = curr.right
                from_ = 2
                if curr is None:
                    return
                continue
            else:
                if curr.left is None and curr.right is None:
                    if from_ is None:
                        self.root = None
                    elif 1 == from_:
                        curr.parent.set_left(None)
                    else:
                        curr.parent.set_right(None)
                elif curr.left is not None and curr.right is None:
                    if from_ is None:
                        self.root = curr.left
                        self.root.set_parent(None)
                        self.root.set_depth(0)
                        # update depth for the subtree
                        self.getInOrder(self.root)
                    elif 1 == from_:
                        curr.parent.set_left(curr.left)
                        # update depth for the subtree
                        self.getInOrder(curr.parent)
                    else:
                        curr.parent.set_right(curr.left)
                        # update depth for the subtree
                        self.getInOrder(curr.parent)
                elif curr.left is None and curr.right is not None:
                    if from_ is None:
                        self.root = curr.right
                        self.root.set_parent(None)
                        self.root.set_depth(0)
                        # update depth for the subtree
                        self.getInOrder(self.root)
                    elif 1 == from_:
                        curr.parent.set_left(curr.right)
                        # update depth for the subtree
                        self.getInOrder(curr.parent)
                    else:
                        curr.parent.set_right(curr.right)
                        # update depth for the subtree
                        self.getInOrder(curr.parent)
                else:
                    # find the node (`r_node`) with the maximum value in the left subtree
                    r_node = curr.left
                    if r_node.right is None:
                        is_left = True
                    else:
                        is_left = False
                    while r_node.right is not None:
                        r_node = r_node.right
                    if is_left:
                        r_node.set_right(curr.right)
                    else:
                        r_node.parent.set_right(r_node.left)
                        r_node.set_left(curr.left)
                        r_node.set_right(curr.right)
                    if from_ is None:
                        self.root = r_node
                        self.root.set_parent(None)
                        self.root.set_depth(0)
                        # update depth for the subtree
                        self.getInOrder(self.root)
                    elif 1 == from_:
                        curr.parent.set_left(r_node)
                        # update depth for the subtree
                        self.getInOrder(curr.parent)
                    else:
                        curr.parent.set_right(r_node)
                        # update depth for the subtree
                        self.getInOrder(curr.parent)
                break
        self.remove(val)
                            
    def getValues(self, depth):
        inorder = self.getInOrder(self.root)
        values = []
        for node in inorder:
            if node.depth > depth:
                continue
            elif node.depth == depth:
                values.append(node.value)
            else:
                if node.left is None:
                    values += [None] * 2**(depth-node.depth-1)
                if node.right is None:
                    values += [None] * 2**(depth-node.depth-1)
        return values
    
    def getInOrder(self, node):
        # in-order traversal
        # update depth
        inorder = []
        def _inorder(node, inorder):
            if node is None:
                return
            elif node.parent is not None:
                node.set_depth(node.parent.depth + 1)
            _inorder(node.left, inorder)
            inorder.append(node)
            _inorder(node.right, inorder)
        _inorder(node, inorder)
        return inorder
    
    def maxDepth(self):
        inorder = self.getInOrder(self.root)
        return max([i.depth for i in inorder])
    
    def __len__(self):
        return self.maxDepth() + 1
    
class DFSTraversal:
    def __init__(self, tree, traversalType):
        self.tree = tree
        self.traversalType = traversalType
        self.changeTraversalType(traversalType)
    
    def changeTraversalType(self, traversalType):
        if not traversalType in DFSTraversalTypes:
            raise TypeError('Invalid traversalType')
        self.traversalType = traversalType
        if traversalType == DFSTraversalTypes.PREORDER:
            self.traversal = self.getPreOrder(self.tree.root)
        elif traversalType == DFSTraversalTypes.INORDER:
            self.traversal = self.getInOrder(self.tree.root)
        elif traversalType == DFSTraversalTypes.POSTORDER:
            self.traversal = self.getPostOrder(self.tree.root)
    
    def getPreOrder(self, node):
        traversal = []
        if node is None:
            return traversal
        s = []
        s.append(node)
        while len(s) > 0:
            node = s.pop()
            traversal.append(node.value)
            if node.right is not None:
                s.append(node.right)
            if node.left is not None:
                s.append(node.left)
        return traversal
    
    def getInOrder(self, node):
        traversal = []
        s = []
        while len(s) > 0 or node is not None:
            if node is not None:
                s.append(node)
                node = node.left
            else:
                node = s.pop()
                traversal.append(node.value)
                node = node.right
        return traversal
    
    def getPostOrder(self, node):
        traversal = []
        s = []
        last_node_visited = None
        while len(s) > 0 or node is not None:
            if node is not None:
                s.append(node)
                node = node.left
            else:
                peek_node = s[-1]
                if peek_node.right is not None and id(last_node_visited) != id(peek_node.right):
                    node = peek_node.right
                else:
                    traversal.append(peek_node.value)
                    last_node_visited = s.pop()
        return traversal
    
    def __getitem__(self, i):
        return self.traversal[i]
        
    def __iter__(self):
        self.index = 0
        return self
    
    def __next__(self):
        try:
            elem = self.traversal[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return elem

Writing TreeTraversal.py


---

## Problem 2: Markov Chains

[Markov Chains](https://en.wikipedia.org/wiki/Markov_chain) are widely used to model and predict discrete events. Underlying Markov chains are Markov processes which make the assumption that the outcome of a future event only depends on the event immediately preceeding it. In this exercise, we will be assuming that weather has Markov properties (e.g. today's weather is dependent only on yesterday's weather). We will use the Markov assumption to create a basic model for predicting weather.

To begin, let's categorize weather into 7 types: ['sunny', 'cloudy', 'rainy', 'snowy', 'windy', 'hailing'].

In the `weather.csv` file accompanying this homework, each row corresponds to one type of weather (in the order given above) and each column is the probability of one type of weather occurring the following day (also in the order given above).

The $ij$th element is the probability that the $j$th weather type occurs after the $i$th weather type. So for example, (1,2) is the probability a cloudy day occurs after a sunny day.

Take a look at the data. Make sure you see how if the previous day was sunny, the following day will have a 0.4 probability of being sunny as well. If the previous day was raining (index $i = 3$), then the following day (index $j$) has a 0.05 probability of being windy ($j = 5$).

### Part 1:  Parse the `.csv` file into a `Numpy` array

In [1]:
#Load CSV file -- hint: you can use np.genfromtxt()

import numpy as np

weather_array = np.genfromtxt('weather.csv', delimiter=',')

print(weather_array)

[[ 0.4   0.3   0.1   0.05  0.1   0.05]
 [ 0.3   0.4   0.1   0.1   0.08  0.02]
 [ 0.2   0.3   0.35  0.05  0.05  0.05]
 [ 0.1   0.2   0.25  0.3   0.1   0.05]
 [ 0.15  0.2   0.1   0.15  0.3   0.1 ]
 [ 0.1   0.2   0.35  0.1   0.05  0.2 ]]


### Part 2:  Create a class called `Markov` that has the following methods:

* `load_data(array)`: loads the Numpy 2D array and stores it as a class variable.
* `get_prob(previous_day, following_day)`: returns the probability of `following_day` weather given `previous_day` weather. 

**Note:** `previous_day` and `following_day` should be passed in string form (e.g. "sunny"), as opposed to an index (e.g. 0). 




In [2]:
class Markov:
    def __init__(self):
        # implement here
        self.weather_types = ['sunny', 'cloudy', 'rainy', 'snowy', 'windy', 'hailing']
        
    def load_data(self, array):
        # implement here
        self.weather_array = array
        return self
    
    def get_prob(self, previous_day, following_day):
        # implement here -- returns a probability
        try:
            i = self.weather_types.index(previous_day)
            j = self.weather_types.index(previous_day)
        except ValueError:
            raise ValueError('Invalid weather.')
        try:
            return self.weather_array[i, j]
        except AttributeError:
            raise AttributeError('Data have not been loaded.')

In [3]:
# example
m = Markov().load_data(weather_array)
print(m.get_prob('sunny', 'sunny'))

0.4


---

## Problem 3: Iterators

Iterators are a convenient way to walk along your Markov chain.

#### Part 1: Using your `Markov` class from Problem 3, write `Markov` as an iterator by implementing the `__iter__()` and `__next__()` methods.

Remember:  
* `__iter__()` should return the iterator object and should be implicitly called when the loop begins
* The `__next()__` method should return the next value and is implicitly called at each step in the loop.

Each 'next' step should be stochastic (i.e. randomly selected based on the relative probabilities of the following day weather types) and should return the next day's weather as a string (e.g. "sunny") rather than an index (e.g. 0).

In [4]:
class Markov:
    def __init__(self):
        # implement here
        self.weather_types = ['sunny', 'cloudy', 'rainy', 'snowy', 'windy', 'hailing']
        self.weather = None
        
    def load_data(self, array):
        # implement here
        self.weather_array = array
        return self
    
    def get_prob(self, previous_day, following_day):
        # implement here -- returns a probability
        try:
            i = self.weather_types.index(previous_day)
            j = self.weather_types.index(previous_day)
        except ValueError:
            raise ValueError('Invalid weather.')
        try:
            return self.weather_array[i, j]
        except AttributeError:
            raise AttributeError('Data have not been loaded.')
    
    def set_weather(self, weather):
        if not weather in self.weather_types:
            raise ValueError('Invalid weather.')
        else:
            self.weather = weather
        return self
    
    def __iter__(self):
        if self.weather is None:
            self.weather = np.random.choice(self.weather_types)
        return self
    
    def __next__(self):
        self.weather = np.random.choice(self.weather_types, \
                                        p=self.weather_array[self.weather_types.index(self.weather)])
        return self.weather

In [5]:
# example

m = Markov().load_data(weather_array).set_weather('cloudy')
m_it = iter(m)
for i in range(5):
    print(next(m_it))

cloudy
sunny
windy
snowy
rainy


#### Part 2: We want to predict what weather will be like in a week for 5 different cities.

Now that we have our `Markov` iterator, we can try to predict what the weather will be like in seven days from now.

Given each city's current weather in the dictionary `city_weather` (see below), simulate what the weather will be like in 7 days from now.  Rather than just producing one prediction per city, simulate 100 such predictions per city and store the most commonly occuring prediction.

In your submission, print a dictionary `city_weather_predictions` that has each city as a key and the most commonly predicted weather as the corresponding value.

**Note**: Don't worry if your values don't seem to make intuitive sense.  We made up the weather probabilities.

In [6]:
city_weather = {
    'New York': 'rainy',
    'Chicago': 'snowy',
    'Seattle': 'rainy',
    'Boston': 'hailing',
    'Miami': 'windy',
    'Los Angeles': 'cloudy',
    'San Fransisco': 'windy'
}

In [7]:
from collections import Counter

def predict(weather, days=7):
    m_it = iter(Markov().load_data(weather_array).set_weather(weather))
    return [next(m_it) for _ in range(days)][-1]
    
city_weather_predictions = {city:Counter([predict(weather) for _ in range(100)]).most_common(1)[0][0] \
                            for city, weather in city_weather.items()}

print(city_weather_predictions)

{'New York': 'cloudy', 'Chicago': 'cloudy', 'Seattle': 'sunny', 'Boston': 'sunny', 'Miami': 'cloudy', 'Los Angeles': 'cloudy', 'San Fransisco': 'cloudy'}
